In [18]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [19]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [24]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.RNN as rnn

class GRU(rnn.RNN):

    def __init__(self, D, H, char2idx, idx2char):
        super().__init__(D, H, char2idx, idx2char)

    def forward(self, X, h_old, train=True):
        m = self.model
        Wz, Wh, Wy = m['Wz'], m['Wh'], m['Wy']
        bz, bh, by = m['bz'], m['bh'], m['by']

        X_one_hot = np.zeros(self.D)
        X_one_hot[X] = 1.
        X_one_hot = X_one_hot.reshape(1, -1)

        # concat: [h, x]
        X = np.column_stack((h_old, X_one_hot))

        # gate: h_prob
        hz, hz_cache = l.fc_forward(X, Wz, bz)
        hz, hz_sigm_cache = l.sigmoid_forward(hz)

        # signal: h_pred
        hh, hh_cache = l.fc_forward(X, Wh, bh)
        hh, hh_tanh_cache = l.tanh_forward(hh)

        # h_next and y_pred
        h = (1. - hz) * h_old + hz * hh
        # h = h_old + hz * (hh - h_old)
        y, y_cache = l.fc_forward(h, Wy, by)

        cache = h_old, X, hz, hz_cache, hz_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache

        if not train:
            y = util.softmax(y)

        return y, h, cache

    def backward(self, y_pred, y_train, dh_next, cache):
        h_old, X, hz, hz_cache, hz_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache = cache

        dy = loss_fun.dcross_entropy(y_pred, y_train)

        dh, dWy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dh_old1 = (1. - hz) * dh

        dhh = hz * dh
        dhh = l.tanh_backward(dhh, hh_tanh_cache)
        dX, dWh, dbh = l.fc_backward(dhh, hh_cache)
        dh_old2 = dX[:, :self.H]

        dhz = hh * dh - h_old * dh
        dhz = l.sigmoid_backward(dhz, hz_sigm_cache)
        dX, dWz, dbz = l.fc_backward(dhz, hz_cache)
        dh_old3 = dX[:, :self.H]

        dh_next = dh_old1 + dh_old2 + dh_old3

        grad = dict(Wz=dWz, Wh=dWh, Wy=dWy, bz=dbz, bh=dbh, by=dby)

        return grad, dh_next

    def _init_model(self, D, C, H):
        Z = H + D

        self.model = dict(
            Wz=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wh=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wy=np.random.randn(H, D) / np.sqrt(D / 2.),
            bz=np.zeros((1, H)),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
        )

In [25]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 10
n_iter = 100000 * 3 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [26]:
import impl.LSTM as lstm
import impl.GRU as gru
import impl.RNN as rnn

# net = nn.LSTM(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.RNN(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
net = GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.GRU(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [ ]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step,
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.3942
 tS af anl zre Sp, oat tha hTto impin rodo welear cxei me the clitioldd J2lpesr Fialgd salapls Ja  n


Iter-2000 loss: 2.7366
ol pureof lht mite  riwrwh- initilalackournbc octtoul's moal 6Oponcerom is ha. on lurarer ao coruily


Iter-3000 loss: 2.3062
rcory r in6 firl whp thil th e coollly god of ming Japcorat milloipmret thce 1xpf ag Covcit, rsith o


Iter-4000 loss: 2.0115
ury. it ricitas ore slomed Indsis mies, porett-and the ffeat war1. R1s"isg ut ofdy fonted pwous lyve


Iter-5000 loss: 1.7911
ctaniWes the atatinof Sentiveled beitargiss 197tho. The cauled y wos cnof us ofd fofith and Horly. w


Iter-6000 loss: 1.6151
ourtr in1 1uthish in Aary the Eutho firodeco smredecthe mar ionld's inttory tsr Puofes ofthind ih th


Iter-7000 loss: 1.4476
a is a gopas If acean
. The Sinok%o Kant filltoving ang the pag thil ioulimamip ane iticy 3m ala , p


Iter-8000 loss: 1.3180
in E3par ort tuobaly Nith lnofured, Chinat is abund Worst-boreas. N2i5s in tyr P and country ic

Iter-25000 loss: 0.6007
riKywClurth 1oth the Immilaries reateres. The courlly. Aithe EOping surled In 1545 Koreation in the 


Iter-26000 loss: 0.5780
 anconercind enjch-lare with oper pountry in Emperor docake op bolaley or hiUllorala destorexth-rake


Iter-27000 loss: 0.5670
hesicedes larsed is and is is meforold imper counsl iepor. Japan is a developmentr Area. The cise em


Iter-28000 loss: 0.5397
vola, wish inxnd in Ebea of ingh Japan cons if the G7, the G8, and the G20 and is the oplltery vetns


Iter-29000 loss: 0.5255
igloxsen fountry 1inh. Japan was calopes and With any. maliol revirea, Asta cary sudged dinthe a ani


Iter-30000 loss: 0.5008
f the Nanicrurte: iunsre fioth largest meity-int wolowed of 19to and In ty the Russo Japan in isto w


Iter-31000 loss: 0.4909
arand inntiy stured Japanea es inctikecily, Appardexth lariestury high tha world'Tasy constraltores 


Iter-32000 loss: 0.4704
kcales imet int the world's fourth-largest. Japan was Toundei loglod ith roved found if

In [ ]:

=========================================================================
Iter-198000 loss: 0.1331
=========================================================================
tary shoguns who ruled in the nainty n mainterd Taiwat lors latulaty und Sunarend thia dudg oonaly r
=========================================================================


=========================================================================
Iter-199000 loss: 0.1386
=========================================================================
th largest imf in the korlation mainlatI un the eipan endand of the Rising Sun". Japan is a stratovo
=========================================================================


=========================================================================
Iter-200000 loss: 0.0987
=========================================================================
hest-ranked Asian countrord with the Since caly towertor Jaty in the country in East Asia. Located i
=========================================================================


Out[11]:
<__main__.GRU at 0x10d053f98>

In [ ]:
=========================================================================
Iter-99000 loss: 0.1890
=========================================================================
 of No20th bex, mfen eped consic th led the fourth leading globan in the name of the Emperor. Japan 
=========================================================================


=========================================================================
Iter-100000 loss: 0.1673
=========================================================================
the early 1shthicaped and the forchest urerlate 19th and the world's fourth-largest economy by purch
=========================================================================


Out[24]:
<__main__.GRU at 0x10d1f7b00>

In [ ]:
=========================================================================
Iter-100000 loss: 0.3913
=========================================================================
the 1pman a inusy rotory surrowho tOkike upporter wor peven des The easty porterized itivecend perce
=========================================================================


Out[7]:
<hipsternet.neuralnet.RNN at 0x7fc2f5182e80>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1615
=========================================================================
the name of the 12th century until 1868, Japan was ruled by successime Wer of 1941, which country is
=========================================================================


Out[16]:
<hipsternet.neuralnet.LSTM at 0x7f67fe0c9978>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1333
=========================================================================
the Coun resilate a in the population. Russided in the OECD and the world's fourth-largest econompic
=========================================================================


Out[10]:
<hipsternet.neuralnet.GRU at 0x7fc2cec1e1d0>